# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770613442108                   -0.53    9.0         
  2   -2.772133158111       -2.82       -1.29    1.0    169ms
  3   -2.772169623712       -4.44       -2.71    1.0    141ms
  4   -2.772170715394       -5.96       -3.66    2.0    190ms
  5   -2.772170722153       -8.17       -4.06    2.0    163ms
  6   -2.772170723005       -9.07       -5.11    1.0    173ms
  7   -2.772170723015      -11.00       -5.75    1.0    147ms
  8   -2.772170723015      -12.29       -6.55    2.0    173ms
  9   -2.772170723015      -13.91       -7.00    1.0    161ms
 10   -2.772170723015   +  -14.31       -7.62    1.0    168ms
 11   -2.772170723015      -14.05       -8.10    1.0    155ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770753215264                   -0.53    9.0         
  2   -2.7

1.7735578254973319

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770796349546                   -0.52    9.0         
  2   -2.772061672156       -2.90       -1.32    1.0    137ms
  3   -2.772083047900       -4.67       -2.44    1.0    180ms
  4   -2.772083342620       -6.53       -3.14    1.0    141ms
  5   -2.772083417439       -7.13       -4.18    2.0    161ms
  6   -2.772083417702       -9.58       -4.57    1.0    168ms
  7   -2.772083417809       -9.97       -5.82    1.0    146ms
  8   -2.772083417811      -11.90       -6.34    2.0    181ms
  9   -2.772083417811      -13.47       -6.69    1.0    160ms
 10   -2.772083417811      -14.01       -8.38    1.0    169ms

Polarizability via ForwardDiff:       1.7725349645830872
Polarizability via finite difference: 1.7735578254973319
